In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b.gguf", device_map="auto",
                                         from_type="gemma", load_eager=True)

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
tokens = tokenizer.encode("The quick brown fox jumps over the lazy dog")

In [5]:
import jax.numpy as jnp
import jax
from micrlhf.llama import LlamaBlock
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
token_array = jnp.asarray([tokens] * 4)
token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
token_array = pz.nx.wrap(token_array, "batch", "seq")
inputs = llama.inputs.from_basic_segments(token_array)
get_resids(inputs)[1][0].value.unwrap("batch", "seq", "embedding")[0]

# jax.Array float32(10, 2048) ≈0.019 ±7.3 [≥-4.6e+02, ≤2.9e+02] zero:276 nonzero:20_204
  Array([[ 5.3911133 ,  0.850647  , -1.4472046 , ..., -0.2844696 ,
           0.        , -0.2844696 ],
         [10.3845215 , -2.8944092 , -3.0932617 , ..., -0.39494324,
           1.977478  , -3.159546  ],
         [13.433594  , -2.3199463 ,  0.74017334, ...,  0.        ,
           2.3530884 , -1.1765442 ],
         ...,
         [10.51709   , -1.1599731 , -4.5515137 , ..., -0.24442291,
           1.9553833 ,  0.97769165],
         [15.466309  , -1.9443359 ,  1.9443359 , ...,  1.9222412 ,
           2.1652832 , -2.6403198 ],
         [12.10791   , -0.85617065,  2.5740356 , ..., -1.1931152 ,
           1.1931152 ,  0.        ]], dtype=float32)

In [6]:
from micrlhf.sampling import sample
texts = sample(llama, tokenizer, "Write me a poem about Machine Learning.",
               batch_size=4, do_sample=True)
texts

  0%|          | 0/64 [00:00<?, ?it/s]

['<bos>Write me a poem about Machine Learning.\n\nMy man Rick said “listen, dawg”,\n\nThe field will be robbed of me\n\nby a customer experience\n\nheaven forbid.\n\nI can’t just come back all major words\n\nin capital letters.\n\nI’m an insecure data scientist\n\nLook at these',
 '<bos>Write me a poem about Machine Learning.\n\nAs you know, downvotes rule the online world. If someone wants to be polite, they want the <em>insight</em>. If anyone is rude, they downvote the comment.\n\nI was delighted to find a machine learning model that fits parents and child chronologically.\n\nCan',
 '<bos>Write me a poem about Machine Learning.\n\nBecause I’m going to the ALT conference in Boston the whole weekend of February 3rd. The entire curriculum for the 2014 edition is written. It’s a balance between learn something new and sharpen something old.\n\nI’m back and',
 '<bos>Write me a poem about Machine Learning. One index really is shining bright!\n\nJapanese students pursue postgraduate studies – An update with figures – Japanese education system chart revised – Report – News – Japanese Education\n\n<b>“Current Vocabulary” of Indian Hindi-speaking region</b>\n\n<b>Facebook Evidence Study</b> – Hilary Khan, Penguin']

In [7]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("alpindale/gemma-2b")

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
import torch
from collections import OrderedDict
for m in model.modules():
    m._forward_hooks = OrderedDict()
model.model.layers[0].register_forward_hook(lambda self, input, output: print(input[0]))
tt = torch.LongTensor(tokens).unsqueeze(0)
model(tt).logits

tensor([[[ 5.3917,  0.8673, -1.4363,  ..., -0.2969,  0.0157, -0.4033],
         [10.3856, -2.8726, -3.0936,  ..., -0.2983,  1.9003, -3.1157],
         [13.4350, -2.3754,  0.7568,  ..., -0.0687,  2.4528, -1.1214],
         ...,
         [10.5182, -1.1270, -4.5299,  ..., -0.3439,  2.0329,  0.9723],
         [15.4680, -1.9224,  1.9666,  ...,  1.8672,  2.1324, -2.6296],
         [12.1092, -0.8121,  2.5301,  ..., -1.1656,  1.1546, -0.0511]]],
       grad_fn=<MulBackward0>)


tensor([[[-26.5950, -13.8833, -18.2747,  ..., -21.2892, -23.2392, -26.5799],
         [-42.4406, -33.3739, -47.8576,  ..., -42.1095, -46.7014, -42.5487],
         [-42.5846, -30.0222, -34.0748,  ..., -40.6030, -38.9625, -42.6937],
         ...,
         [-41.7852, -19.4810, -38.3410,  ..., -38.2935, -41.6302, -41.8573],
         [-39.3097, -18.4143, -34.3758,  ..., -36.4977, -40.6678, -39.3800],
         [-42.2921, -23.0559, -36.2486,  ..., -38.2521, -37.0666, -42.3374]]],
       grad_fn=<UnsafeViewBackward0>)